In [1]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [17]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'

# j_state = catalog.id['Jmag'] > 14
# subset_group2 = dc.new_subset_group('Jmag > 14', j_state)


In [18]:
import glue.utils.array as gua
import glue.core.data as gcd
import pandas as pd
from pandas import DataFrame
import numpy as np

In [19]:
# # Defines a subset and runs statistics using compute_statistic for the subset
# # Saves data using a pandas dataframe called my_pandas_data

# # Data that subset is pulled from
# data = dc[0]

# # Define state and subset
# state1 = data.id['Jmag'] > 14
# subset1 = data.new_subset(state1, label='Jmag > 14')

# # Same arrays as with full data
# mean_array = []
# median_array = []
# min_array = []
# max_array = []
# sum_array = []
# name_array = []
# tables = []

# headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

# print(data.label, 'subset1')
# print() 
# for j in range (0, len(data.components)):
#     name = data.components[j].label # Get the name of each component
#     name_array.append(name) # add to the name array to build the table
#     mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
#     median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
#     min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
#     max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
#     sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
# column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

# my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

In [25]:
from PyQt5.QtCore import QAbstractItemModel, pyqtSignal, QSize, QFile, QIODevice, QModelIndex, Qt, pyqtSlot, QVariant, QItemSelectionModel
from PyQt5.QtWidgets import QSizePolicy, QTreeView, QAbstractItemView, QApplication, QLabel, QTreeView, QComboBox, QCheckBox, QWidget, QPushButton, QHBoxLayout, QFrame, QTableView, QGroupBox, QDialog, QVBoxLayout, QLabel, QGridLayout
from PyQt5 import QtCore, QtWidgets, QtGui
from PyQt5.QtGui import QStandardItemModel, QStandardItem
from decimal import getcontext, Decimal
from IPython.display import display, HTML
from PyQt5.QtGui import *
import datetime
import sys

class pandasModel(QtCore.QAbstractTableModel):
    # Set up the data in a form that allows it to be added to qt widget
    def __init__(self, df, parent=None):
        QtCore.QAbstractTableModel.__init__(self, parent)
        self.data_frame = df
        super(pandasModel, self).__init__(parent)      

    def rowCount(self, parent=None):
        return len(self.data_frame.values)

    def columnCount(self, parent=None):
        return self.data_frame.columns.size

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return QVariant(str(
                    self.data_frame.values[index.row()][index.column()]))
        return QVariant()
    
    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return QVariant(self.data_frame.columns[col])
        if orientation == Qt.Vertical and role == Qt.DisplayRole:
            return QVariant(self.data_frame.index[col])
        return QVariant()  
    
    def sort(self, column, order):
        colname = self.data_frame.columns.tolist()[column]
        self.layoutAboutToBeChanged.emit()
        self.data_frame.sort_values(colname, ascending= order == QtCore.Qt.AscendingOrder, inplace=True)
        self.data_frame.reset_index(inplace=True, drop=True)
        self.layoutChanged.emit()

class StatsGui(QWidget):
    ''' 
    This class accepts a glue data collection object, and builds an interactive window
    to display basic statistics (e.g. mean, median, mode) about each dataset
    '''
    
    clicked = pyqtSignal()
    keyPressed = pyqtSignal()
    
    def __init__(self,dc):
        
        # Initialize the object as a QWidget
        QWidget.__init__(self)
        
#         # Set up dynamic resizing
#         sizePolicy = QSizePolicy(QSizePolicy.Preferred, QSizePolicy.Preferred)
#         sizePolicy.setHeightForWidth(True)
#         self.setSizePolicy(sizePolicy)
    
        #Save the datacollection object as an attribute of class StatsGui
        self.dc=dc
        
        #Fix the size of the main GUI window (for now)
        self.setMinimumSize(800, 630)
        
        #Set the title of the main GUI window
        self.setWindowTitle('Statistics')
        
        #Set up tree view and fix it to the top half of the window
        self.treeview = QTreeView(self)
        self.treeview.setGeometry(50, 25, 400, 200) 
#         self.treeview.setSizePolicy(sizePolicy)
    
        
        #Every time a selection is made in the tree, perform the functionality in 'myPressedEvent'
        self.treeview.pressed.connect(self.myPressedEvent)

        #Set the default clicking behavior to be row selection
        self.treeview.setSelectionBehavior(QAbstractItemView.SelectRows)
        model = QStandardItemModel()
        model.setHorizontalHeaderLabels([''])

        self.treeview.setModel(model)
        self.treeview.setUniformRowHeights(True)
        
        #Allow the user to select multiple rows at a time 
        self.treeview.setSelectionMode(QAbstractItemView.MultiSelection)
    
        # populate the tree
        # Make all the datasets be parents, and make it so they are not selectable
        parent_data = QStandardItem('{}'.format('Data'))
        parent_data.setEditable(False)
        parent_data.setSelectable(False)
        
        for i in range(0,len(dc)):
            parent = QStandardItem('{}'.format(self.dc.labels[i]))
            parent.setEditable(False)
            parent.setSelectable(False)
            
            # Make all the data components be children, nested under their parent
            for j in range(0,len(self.dc[i].components)):
                child=QStandardItem('{}'.format(str(self.dc[i].components[j])))
                child.setEditable(False)
                parent.appendRow(child)
                                    
            parent_data.appendRow(parent)
                                    
            #Add the parents with their children to the QStandardItemModel
        model.appendRow(parent_data)
        
        parent_subset = QStandardItem('{}'.format('Subsets')) 
        parent_subset.setEditable(False)
        parent_subset.setSelectable(False)
                                    
        # Set up the subsets as Subsets > choose subset > choose data set > choose component
      
        for j in range(0, len(self.dc.subset_groups)):
            grandparent = QStandardItem('{}'.format(self.dc.subset_groups[j].label))
            grandparent.setEditable(False)
            grandparent.setSelectable(False)
            
            for i in range(0, len(self.dc)):
                parent = QStandardItem('{}'.format(self.dc.subset_groups[j].label) + ' (' + '{}'.format(self.dc[i].label) + ')')
                parent.setEditable(False)
                parent.setSelectable(False)
                    
                for k in range(0, len(self.dc[i].components)):
                    child = QStandardItem('{}'.format(str(self.dc[i].components[k])))
                    child.setEditable(False)
                    parent.appendRow(child)
                            
                grandparent.appendRow(parent) 
                    
            parent_subset.appendRow(grandparent)
                                    
        model.appendRow(parent_subset)
            
        # Set up the combo box for users to choose the number of significant figures in the table
        self.sigfig = QComboBox(self)
        self.sigfig.addItems(["Default = 5"])
        self.sigfig.addItems(list('12345678'))
        self.sigfig.model().item(0).setEnabled(False)
        self.sigfig.move(500, 50)
        self.sigfig.currentIndexChanged.connect(self.sigchange)
        
        self.siglabel = QLabel(self)
        self.siglabel.setText('Number of significant figures:')
        self.siglabel.move(500, 25)
        
        # Set up select all and deselect all buttons
        self.all = QCheckBox(self)
        self.all.setText('Select all')
        self.all.stateChanged.connect(self.allClicked)
        self.all.move(500, 100)
        
        # Set default significant figures to 5
        getcontext().prec = 5
            
        #################Set up the QTableView Widget#############################
        self.table = QTableView(self)
        self.table.setAlternatingRowColors(True)
        self.table.setSortingEnabled(True)
        self.table.resizeColumnsToContents()
        self.table.resizeRowsToContents()
        self.table.verticalHeader().setVisible(False)
        
        #Move the table widget to the bottom of the GUI window
        self.table.setGeometry(50, 250, 700, 300) 
        
        #Set the table headings -- I have added Dataset and Component Columns. 
        self.headings = ('Subset', 'Dataset', 'Component','Mean', 'Median', 'Minimum', 'Maximum', 'Sum')   
        self.data_frame = pd.DataFrame(columns=self.headings) 
        self.data_accurate = pd.DataFrame(columns=self.headings)
        self.model = pandasModel(self.data_frame)

        self.table.setModel(self.model)
        
        # Export to file button
        self.export = QPushButton(self)
        self.export.setText('Export to file')
        self.export.clicked.connect(self.exportToFile)
        self.export.move(350, 580)

        
    def myPressedEvent (self, currentQModelIndex):
        ''' 
        Every time a row (or rows) in the tree view is clicked:
        if it is selected, add it to the table
        if it is deselected, remove it from the table
        ''' 
        data_i = currentQModelIndex.parent().row()
        comp_i = currentQModelIndex.row()
        subset_i = currentQModelIndex.parent().parent().row()

        # Get the indexes of all the selected components
        self.indexes=self.treeview.selectionModel().selectedRows()

        # If nothing is selected clear the stats from the table
        if len(self.indexes) == 0:
            self.data_frame = pd.DataFrame(columns=self.headings)          
        
        # See if the current index is in the selected components
        if currentQModelIndex in self.indexes:
            if currentQModelIndex.parent().parent().parent().row() == 1:
                print(currentQModelIndex.parent().parent().parent().row(), currentQModelIndex.parent().parent().row(), currentQModelIndex.parent().row(), currentQModelIndex.row())
                self.runSubsetStats(subset_i, data_i, comp_i)
            else:
                print(currentQModelIndex.parent().parent().row(), currentQModelIndex.parent().row(), currentQModelIndex.row())
            # If the current index is selected, call runStats to append it to the data_frame
                self.runDataStats(currentQModelIndex.parent().row(), currentQModelIndex.row())
        
        # If it is not in the selected components, drop it from the table
        else:
            # Find the index of the unchecked element in the table and remove that row
            
            if currentQModelIndex.parent().parent().parent().row() == 1:
                try:
                    idx_c = np.where(self.data_frame['Component'] == self.dc[data_i].components[comp_i].label)
                    idx_d = np.where(self.data_frame['Dataset'] == self.dc[data_i].label)
                    idx_s = np.where(self.data_frame['Subset'] == self.dc[data_i].subsets[subset_i].label)
                    print('idx_c', idx_c)
                    print('idx_d', idx_d)
                    print('idx_s', idx_s)
                    idx1 = np.intersect1d(idx_c, idx_d)
                    print('idx1', idx1)
                    idx2 = np.intersect1d(idx1, idx_s)
                    print('idx2', idx2)
                    self.data_frame = self.data_frame.drop(idx2)
                except:
                    pass
                
            else:
                try:
                # Find the index in the table of the unchecked element, if it's in the table
                
                    # Find the matching component and dataset indices and intersect them to get the unique index
                    idx_c = np.where(self.data_frame['Component'] == self.dc[data_i].components[comp_i].label)
                    idx_d = np.where(self.data_frame['Dataset'] == self.dc[data_i].label)
                    idx_s = np.where(self.data_frame['Subset'] == '--')
                    idx1 = np.intersect1d(idx_c, idx_d)
                    idx2 = np.intersect1d(idx1, idx_s)
                    self.data_frame = self.data_frame.drop(idx2)
                except:
                    pass
            
        model = pandasModel(self.data_frame)
        self.table.setModel(model)
        self.table.setSortingEnabled(True)
        self.table.setShowGrid(False)  
    
    def runDataStats (self, data_i, comp_i):
        '''
        Runs statistics for the component comp_i of data set data_i
        '''

        subset_label = "--"
        data_label = dc[data_i].label   
        comp_label = self.dc[data_i].components[comp_i].label # add to the name array to build the table
        
        # Find the true values and save them in a DataFrame called self.data_accurate
        
        mean_val = self.dc[data_i].compute_statistic('mean', self.dc[data_i].components[comp_i])
        median_val = self.dc[data_i].compute_statistic('median', self.dc[data_i].components[comp_i])     
        min_val = self.dc[data_i].compute_statistic('minimum', self.dc[data_i].components[comp_i])     
        max_val = self.dc[data_i].compute_statistic('maximum', self.dc[data_i].components[comp_i])    
        sum_val = self.dc[data_i].compute_statistic('sum', self.dc[data_i].components[comp_i])
        
        column_data = np.asarray([[subset_label], [data_label], [comp_label], [mean_val], [median_val], [min_val], [max_val], [sum_val]]).transpose()
        column_df = pd.DataFrame(column_data, columns = self.headings)
        self.data_accurate = self.data_accurate.append(column_df, ignore_index = True)        
        
        # Round the values according to the sig figs set by the user
        mean_val = Decimal(float(mean_val)) * Decimal(1)
        median_val = Decimal(float(median_val)) * Decimal(1)      
        min_val = Decimal(float(min_val)) * Decimal(1)     
        max_val = Decimal(float(max_val)) * Decimal(1)    
        sum_val = Decimal(float(sum_val)) * Decimal(1)
        
        # Create the column data array and append it to the data frame
        column_data = np.asarray([[subset_label], [data_label], [comp_label], [mean_val], [median_val], [min_val], [max_val], [sum_val]]).transpose()
        column_df = pd.DataFrame(column_data, columns = self.headings)
        self.data_frame = self.data_frame.append(column_df, ignore_index = True)
    
    def runSubsetStats (self, subset_i, data_i, comp_i):
        '''
        Runs statistics for the subset subset_i with respect to the component comp_i of data set data_i
        '''

        subset_label = dc[data_i].subsets[subset_i].label
        data_label = dc[data_i].label   
        comp_label = self.dc[data_i].components[comp_i].label # add to the name array to build the table
        
        # Find the true values and save them in a DataFrame called self.data_accurate
        
        mean_val = self.dc[data_i].compute_statistic('mean', self.dc[data_i].subsets[subset_i].components[comp_i], subset_state=self.dc[data_i].subsets[subset_i].subset_state)
        median_val = self.dc[data_i].compute_statistic('median', self.dc[data_i].subsets[subset_i].components[comp_i], subset_state=self.dc.subset_groups[subset_i].subset_state)       
        min_val = self.dc[data_i].compute_statistic('minimum', self.dc[data_i].subsets[subset_i].components[comp_i], subset_state=self.dc.subset_groups[subset_i].subset_state)       
        max_val = self.dc[data_i].compute_statistic('maximum', self.dc[data_i].subsets[subset_i].components[comp_i], subset_state=self.dc.subset_groups[subset_i].subset_state)      
        sum_val = self.dc[data_i].compute_statistic('sum', self.dc[data_i].subsets[subset_i].components[comp_i], subset_state=self.dc.subset_groups[subset_i].subset_state) 
        
        column_data = np.asarray([[subset_label], [data_label], [comp_label], [mean_val], [median_val], [min_val], [max_val], [sum_val]]).transpose()
        column_df = pd.DataFrame(column_data, columns = self.headings)
        self.data_accurate = self.data_accurate.append(column_df, ignore_index = True)        
        
        # Round the values according to the sig figs set by the user
        mean_val = Decimal(float(mean_val)) * Decimal(1)
        median_val = Decimal(float(median_val)) * Decimal(1)      
        min_val = Decimal(float(min_val)) * Decimal(1)     
        max_val = Decimal(float(max_val)) * Decimal(1)    
        sum_val = Decimal(float(sum_val)) * Decimal(1)
        
        # Create the column data array and append it to the data frame
        column_data = np.asarray([[subset_label], [data_label], [comp_label], [mean_val], [median_val], [min_val], [max_val], [sum_val]]).transpose()
        column_df = pd.DataFrame(column_data, columns = self.headings)
        self.data_frame = self.data_frame.append(column_df, ignore_index = True)    
    
    def sigchange(self, i):
        # Set the number of significant figures according to what the user selects
        getcontext().prec = i
        
        # Retrospectively change the number of significant figures in the table
        
        data_labels = self.data_frame['Dataset']
        comp_labels = self.data_frame['Component']
        subset_labels = self.data_frame['Subset']
        
        mean_vals = []
        median_vals = []
        min_vals = []
        max_vals = []
        sum_vals = []
        
        # Get the values from the self.data_accurate array and append them
        for i in range (0, len(self.data_frame)):
            mean_vals.append(Decimal(self.data_accurate['Mean'][i]) * Decimal(1))
            median_vals.append(Decimal(self.data_accurate['Median'][i]) * Decimal(1))
            min_vals.append(Decimal(self.data_accurate['Minimum'][i]) * Decimal(1))
            max_vals.append(Decimal(self.data_accurate['Maximum'][i]) * Decimal(1))
            sum_vals.append(Decimal(self.data_accurate['Sum'][i]) * Decimal(1))
            
        column_data = np.asarray([subset_labels, data_labels, comp_labels, mean_vals, median_vals, min_vals, max_vals, sum_vals]).transpose()
        self.data_frame = pd.DataFrame(column_data, columns=self.headings)
        model = pandasModel(self.data_frame)
        self.table.setModel(model)
        self.table.setSortingEnabled(True)
        self.table.setShowGrid(False) 
        
    def allClicked(self):
        # Select all components of the treeview if checked and fill the table with newly checked items
        if self.all.isChecked() == True:
            original_idx = self.treeview.selectionModel().selectedRows()
            self.treeview.selectAll()
            end_idx=self.treeview.selectionModel().selectedRows()
            for index in end_idx:
                if index not in original_idx:
                    if index.parent().parent().parent().row() != 1:
                        self.runDataStats(index.parent().row(), index.row())
                    else:
                        self.runSubsetStats(index.parent().parent().row(), index.parent().row(), index.row())
            
        else:
            # Deselect and clear the table if unchecked
            self.treeview.clearSelection()
            self.data_frame = pd.DataFrame(columns = self.headings)
        
        # Set the table to display the correct data frame
        model = pandasModel(self.data_frame)
        self.table.setModel(model)
        self.table.setSortingEnabled(True)
        self.table.setShowGrid(False) 
        
    def exportToFile(self):
        now = datetime.datetime.now()
        file_name = 'stats' + '_' + str(now.month) + '_' + str(now.day) + '_' + str(now.year) + '_' + str(now.hour) + ':' + str(now.minute) + '.csv'
        self.data_frame.to_csv(file_name, index=False)
  
    def sizeHint(self):
        return QSize(400, 600)
    
    def heightForWidth(self, width):
        return width * 1.5

app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
ex = StatsGui(dc)
ex.show()
sys.exit(app.exec_())

QApplication instance already exists: <PyQt5.QtWidgets.QApplication object at 0x1515b1ae58>


SystemExit: 0

/Users/laurachapman/anaconda/envs/glueviz-dev/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Builds a widget that resizes well 

import sys
from PyQt5.QtCore import pyqtSignal, QSize, Qt
from PyQt5.QtGui import *

class MyWidget(QWidget):

    clicked = pyqtSignal()
    keyPressed = pyqtSignal()
    
    def __init__(self, parent = None):
    
        QWidget.__init__(self, parent)
        self.color = QColor(0, 0, 0)
        sizePolicy = QSizePolicy(QSizePolicy.Preferred, QSizePolicy.Preferred)
        sizePolicy.setHeightForWidth(True)
        self.setSizePolicy(sizePolicy)
    
    def paintEvent(self, event):
    
        painter = QPainter()
        painter.begin(self)
        painter.fillRect(event.rect(), QBrush(self.color))
        painter.end()
    
    def keyPressEvent(self, event):
    
        self.keyPressed.emit(event.text())
        event.accept()
    
    def mousePressEvent(self, event):
    
        self.setFocus(Qt.OtherFocusReason)
        event.accept()
    
    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
        
            self.color = QColor(self.color.green(), self.color.blue(),
                                127 - self.color.red())
            self.update()
            self.clicked.emit()
            event.accept()
    
    def sizeHint(self):
    
        return QSize(800, 600)

    def heightForWidth(self, width):
    
        return width * 2.5

app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
    
window = QWidget()
    
mywidget = MyWidget()
label = QLabel()
    
# mywidget.clicked.connect(label.clear)
# mywidget.keyPressed.connect(label.setText)
    
layout = QVBoxLayout()
layout.addWidget(mywidget, 0, Qt.AlignCenter)
layout.addWidget(label)
window.setLayout(layout)
    
window.show()
sys.exit(app.exec_())